In [43]:
print('Hi, Jupyter!')

Hi, Jupyter!


In [ ]:
!pip3 install pymystem3==0.1.10
!pip3 install pymorphy2[fast]
!pip3 install razdel
!pip3 install gensim
!pip3 install nltk
!pip3 install rusenttokenize
!pip3 install regex

  Using cached https://files.pythonhosted.org/packages/51/56/57e550b53587719e92330a79c7c0f555402d953b00700efae6d5ca53cdef/pymystem3-0.1.10-py3-none-any.whl
    100% |████████████████████████████████| 61kB 494kB/s ta 0:00:01
  Using cached https://files.pythonhosted.org/packages/bc/a9/01ffebfb562e4274b6487b4bb1ddec7ca55ec7510b22e4c51f14098443b8/chardet-3.0.4-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/a2/38/928ddce2273eaa564f6f50de919327bf3a00f091b5baba8dfa9460f3a8a8/idna-2.10-py2.py3-none-any.whl
    100% |████████████████████████████████| 163kB 703kB/s ta 0:00:01
    100% |████████████████████████████████| 143kB 658kB/s ta 0:00:01
  Using cached https://files.pythonhosted.org/packages/07/57/b2ff2fae3376d4f3c697b9886b64a54b476e1a332c67eee9f88e7f1ae8c9/pymorphy2-0.9.1-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/6a/84/ff1ce2071d4c650ec85745766c0047ccc3b5036f1d03559fd46bb38b5eeb/DAWG_Python-0.7.2-py2.py3-none-any.whl
  Using cach

In [22]:
text = open('Рабочий стол/zhivago.txt','r').read()
#print(text)

In [28]:
import string
from gensim.utils import tokenize
from gensim.summarization.textcleaner import split_sentences
from rusenttokenize import ru_sent_tokenize
from razdel import sentenize
from razdel import tokenize as razdel_tokenize
from nltk import sent_tokenize
from nltk.tokenize import word_tokenize, wordpunct_tokenize
from pymorphy2 import MorphAnalyzer
from pymystem3 import Mystem
from nltk.corpus import stopwords
from string import punctuation
import re, os, json


In [29]:
# героически пытаюсь пользоваться регулярками
# первый этап очистки, убираю теги
clean_text_tags = re.sub(r'<[^>]+>', ' ', text)
# убираю пробелы
clean_text_space = re.sub('  +', ' ', clean_text_tags)
# убираю английский слова
clean_text_eng = re.sub(r'[A-za-z].+', ' ', clean_text_space)
#print(clean_text_eng)
# убираю ненужные цифры в конце и в начале после англ слов, я читер года
# простите!
clean_text_nums = re.sub(r'\n(([+]?\d+)|/|(/+?|\+|\+?\d.*))|(17.04.2008)|(\/9)|(\+)|(\/)', ' ', clean_text_eng)
#print(clean_text_nums)
# опять убираю пробелы
clean_text_spaces = re.sub('  +', ' ', clean_text_nums)
#print(clean_text_spaces)

In [30]:
# делю текст на предложения
# to_sent = ru_sent_tokenize(clean_text_spaces)# Something went wrong while tokenizing!
# я испугалась, он начал виснуть, сделала по другому
sent = re.split('(?<=[\.?!]) +(?=[А-ЯЁ])', clean_text_spaces.replace('\n', ' '))
sentences = []
for s in sent:
    s.replace('\xa0',' ')
    s = re.sub('  +', ' ', s)
    #print(s)
    # вот тут лежат предложения, если что, их всегда можно достать
    s = s.lower()
    sentences.append(s)
    #print(s)
#print(sentences[:10])
# делю исходный чистый текст на токены, убираю токены-пункт знаки
string.punctuation += '«»—…“”'
string.punctuation += '–'
# собственно, вот список токенов-слов
tokens = []
tokenized_text = razdel_tokenize(clean_text_spaces.lower())
for token in tokenized_text:
    if token.text not in string.punctuation:
        tokens.append(token.text)
        #print(token.text)

In [31]:
# давайте искать повторяющиесяся предложения
repeating_sentences = []
for sent in sentences:
    if sentences.count(sent)>1:
        repeating_sentences.append(sent)
print(repeating_sentences[:10])

['единственно живое и яркое в вас – это то, что вы жили в одно время со мной и меня знали».', 'весной в несколько дней лес преображается, подымается до облаков, в его покрытых листьями дебрях можно затеряться, спрятаться.', 'это превращение достигается движением, по стремительности превосходящим движения животных, потому что животное не растет так быстро, как растение, и которого никогда нельзя подсмотреть.', 'лес не передвигается, мы не можем его накрыть, подстеречь за переменою места.', 'мы всегда застаем его в неподвижности.', 'и в такой же неподвижности застигаем мы вечно растущую, вечно меняющуюся, неуследимую в своих превращениях жизнь общества, историю.', 'толстой не довел своей мысли до конца, когда отрицал роль зачинателей за наполеоном, правителями, полководцами.', 'он думал именно то же самое, но не договорил этого со всею ясностью.', 'потом пришла война, разруха, и долго-долго о тебе ни слуху не было, ни духу.', 'и через много-много лет твой голос вновь меня встревожил.']


In [32]:
# теперь разбираемся с токенами, нам надо найти
#самый частотный токен в тексте длиннее 6 символов
tokens_dict = {}
for token in tokens:
    if len(token)>6:
        tokens_dict.setdefault(token, 0)
        tokens_dict[token] += 1
# сортируем словарь        
dict_sort = sorted(tokens_dict.items(),key=lambda i: i[1], reverse = True)
print(dict_sort[0])
       
        

('андреевич', 289)


In [33]:
# делаем стемминг
# у нас сейчас получится частотный список, потому что я вспомнила про Counter
# из него выберем какие нить частотные слова и проверим, что как
from collections import Counter
from nltk.stem.snowball import SnowballStemmer
stems_dict = {}
# не забываем копировать из конспекта
stemmer = SnowballStemmer('russian')
stems = [(word, stemmer.stem(word)) for word in tokens]
also_stems = set(stems)
freq_list = []
for i, stem in enumerate(also_stems):
    freq_list.append(stem[1])
common = Counter(freq_list)
print(common.most_common(10))


[('пуст', 31), ('цел', 26), ('прост', 25), ('след', 25), ('сдела', 23), ('оста', 22), ('прав', 22), ('бел', 21), ('продолжа', 21), ('ход', 21)]


In [34]:
# давайте возьмем первые 3 стемма
first = []
second = []
third = []
for stem in also_stems:
    if stem[1] == 'пуст':
        first.append(stem)
print(first[:10])        
for stem in also_stems:
    if stem[1] == 'цел':
        second.append(stem)
print(second[:10])        
for stem in also_stems:
    if stem[1] == 'след':
        third.append(stem)
print(third[:10])    

# как я поняла, из того, что в итоге распечаталось, стеммер пихает, например, в тот же "пуст" что угодно,
# хотя так не надо делать; было бы правильнее, если бы он не откусывал ь от слова пусть, и, например,
# оставил уй у целуй

[('пустит', 'пуст'), ('пустующую', 'пуст'), ('пустая', 'пуст'), ('пустился', 'пуст'), ('пустом', 'пуст'), ('пустому', 'пуст'), ('пусто', 'пуст'), ('пустив', 'пуст'), ('пустишься', 'пуст'), ('пусти', 'пуст')]
[('целого', 'цел'), ('целях', 'цел'), ('целой', 'цел'), ('целая', 'цел'), ('цел', 'цел'), ('целых', 'цел'), ('цела', 'цел'), ('целы', 'цел'), ('целуйте', 'цел'), ('целые', 'цел')]
[('следом', 'след'), ('следующей', 'след'), ('следите', 'след'), ('след', 'след'), ('следующих', 'след'), ('следующий', 'след'), ('следила', 'след'), ('следуют', 'след'), ('следует', 'след'), ('следам', 'след')]


In [35]:
# теперь посмотрим, что там по неизменным словам
without_changes = []
for stem in stems:
        if len(stem[0]) > 4:
            if stem[0] == stem[1]:
                without_changes.append(stem)
common = Counter(without_changes)
print(common.most_common(15))
# и вот мы набдюдаем слова типа может, будет,которые надо было бы укоротить, но не получилось

[(('андреевич', 'андреевич'), 289), (('доктор', 'доктор'), 226), (('перед', 'перед'), 172), (('может', 'может'), 162), (('через', 'через'), 144), (('будет', 'будет'), 141), (('сейчас', 'сейчас'), 127), (('человек', 'человек'), 112), (('вдруг', 'вдруг'), 109), (('поезд', 'поезд'), 65), (('видел', 'видел'), 53), (('назад', 'назад'), 49), (('гордон', 'гордон'), 47), (('город', 'город'), 45), (('хотел', 'хотел'), 45)]


In [39]:
# я не успела сделать 4 задание, потому что отложила его на ненаступившее потом, но частично успела 5, вот оно:
# итак, я сравниваю майстем и пайморфи
mystem = Mystem()
morph = MorphAnalyzer()
#print(mystem.lemmatize(clean_text_spaces)[100:500])
#  мне не понравилось, что тут к словам цепляется ',\xa0– '? если распечатывать кучу всего
# {'analysis': [{'lex': 'фарисейский', 'wt': 1, 'gr': 'A=мн,кр'}], 'text': 'фарисейски'} - вот тут оно вообще прилагательное
# {'analysis': [{'lex': 'творчески', 'wt': 0.9986144172, 'gr': 'ADV='}], 'text': 'творчески'} - это норм, как и у пайморфи
# {'analysis': [{'lex': 'окружающие', 'wt': 0.6348522644, 'gr': 'S,мн,од=(пр|вин|род)'}], 'text': 'окружающих'} - а тут нормально

[' ', 'который', ' ', 'убедительно', ' ', 'свидетельствовать', ' ', 'о', ' ', 'чудо', ', ', 'чем', ' ', 'весь', ' ', 'размышление', ' ', 'доктор', ' ', 'и', ' ', 'обобщение', ' ', 'автор', '. ', 'человек', ', ', 'который', ' ', 'так', ' ', 'писать', ', ', 'бесконечно', ' ', 'много', ' ', 'переживать', ' ', 'и', ' ', 'передумывать', ', ', 'и', ' ', 'главный', ' ', 'его', ' ', 'чувство', ' ', 'на', ' ', 'свет', ' - ', 'восхищенный', ' ', 'умиление', ' ', 'и', ' ', 'слезный', ' ', 'сострадание', '; ', 'конечно', ', ', 'быть', ' ', 'в', ' ', 'его', ' ', 'мир', ' ', 'место', ' ', 'и', ' ', 'презрение', ', ', 'и', ' ', 'холодный', ' ', 'отстранение', ' - ', 'но', ' ', 'не', ' ', 'в', ' ', 'они', ' ', 'суть', '. ', 'роман', ' ', 'пастернак', ' - ', 'оплакивание', ' ', 'прежний', ' ', 'заблуждение', ' ', 'и', ' ', 'их', ' ', 'жертва', '; ', 'тот', ', ', 'кто', ' ', 'не', ' ', 'разделять', ' ', 'молитвенный', ' ', 'восторг', ' ', 'перед', ' ', 'мир', ', ', 'достойный', ' ', 'прежде', ' ', 'все'

In [42]:
words_analized = mystem.analyze(clean_text_spaces)
print(words_analized[1000:1500])

[{'analysis': [{'lex': 'гибель', 'wt': 1, 'gr': 'S,жен,неод=(пр,ед|вин,мн|дат,ед|род,ед|им,мн)'}], 'text': 'гибели'}, {'text': '. '}, {'analysis': [{'lex': 'в', 'wt': 0.9999917878, 'gr': 'PR='}], 'text': 'В'}, {'text': ' '}, {'analysis': [{'lex': 'ход', 'wt': 0.9998553034, 'gr': 'S,муж,неод=пр,ед'}], 'text': 'ходе'}, {'text': ' '}, {'analysis': [{'lex': 'этот', 'wt': 1, 'gr': 'APRO=(пр,мн|род,мн|вин,мн,од)'}], 'text': 'этих'}, {'text': ' '}, {'analysis': [{'lex': 'год', 'wt': 0.9954891636, 'gr': 'S,муж,неод=род,мн'}], 'text': 'лет'}, {'text': ' '}, {'analysis': [{'lex': 'немногие', 'wt': 0.1945949197, 'gr': 'SPRO,мн=дат'}], 'text': 'немногим'}, {'text': ' '}, {'analysis': [{'lex': 'удаваться', 'wt': 1, 'gr': 'V,нп=прош,ед,изъяв,сред,сов'}], 'text': 'удалось'}, {'text': ' '}, {'analysis': [{'lex': 'оставаться', 'wt': 1, 'gr': 'V,нп=инф,сов'}], 'text': 'остаться'}, {'text': ' '}, {'analysis': [{'lex': 'верный', 'wt': 1, 'gr': 'A=твор,мн,полн'}], 'text': 'верными'}, {'text': ' '}, {'analy

In [25]:
# основная функция - pymorphy.parse
morph = MorphAnalyzer()
words_analized = [morph.parse(token) for token in word_tokenize(clean_text_spaces)]
#for item in words_analized:
        #print(item, '\n')
    
#это наречие, а не существительное!!!
# хотя в основном он нормально наречия определяет, напр, сначала - все хорошо, в наречиях на о тоже все хорошо, насколько я могла заметить, может я что то пропустила.
# может, дело в окончании, пайморфи путается из за И
#[Parse(word='фарисейски', tag=OpencorporaTag('NOUN,inan,masc,Geox plur,nomn'), normal_form='фарисейск', score=0.3879310344827587, methods_stack=((DictionaryAnalyzer(), 'ейски', 74, 6), (UnknownPrefixAnalyzer(score_multiplier=0.5), 'фарис'))), Parse(word='фарисейски', tag=OpencorporaTag('NOUN,inan,masc,Geox plur,accs'), normal_form='фарисейск', score=0.3879310344827587, methods_stack=((DictionaryAnalyzer(), 'ейски', 74, 9), (UnknownPrefixAnalyzer(score_multiplier=0.5), 'фарис'))), Parse(word='фарисейски', tag=OpencorporaTag('ADVB'), normal_form='фарисейски', score=0.22413793103448276, methods_stack=((FakeDictionary(), 'фарисейски', 3, 0), (KnownSuffixAnalyzer(min_word_length=4, score_multiplier=0.5), 'ейски')))] 

# а тут не путается..[Parse(word='творчески', tag=OpencorporaTag('ADVB'), normal_form='творчески', score=1.0, methods_stack=((DictionaryAnalyzer(), 'творчески', 3, 0),))] 
# [Parse(word='окружающих', tag=OpencorporaTag('ADJF plur,gent'), normal_form='окружающий', score=0.09090909090909091, methods_stack=((DictionaryAnalyzer(), 'окружающих', 213, 21),))
# это должно быть существительное


In [ ]:
#я так понимаю, они стоят друг друга в плане ошибок, кто что; хотя я отдаю себе отчет в том, 
#что я руками-глазами не могла проверить все,
#например, я не проверяла союзы, частицы и предлоги - их даже люди иногда путают,
# не удивлюсь, если пайморфи, например, тоже запутается